In [1]:
!nvcc -V
!echo
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git
!pip install -r /content/Swin-Transformer-Object-Detection/requirements.txt

In [ ]:
!pip install openmim
!mim install mmdet

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

In [ ]:
%cd Swin-Transformer-Object-Detection/

In [ ]:
import sys
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, set_random_seed, init_detector
from mmdet.datasets import build_dataloader, build_dataset, replace_ImageToTensor
from mmdet.utils import collect_env, get_root_logger

import torch
import time
import json

from pprint import pprint
import os

In [ ]:
classes = (
    "ulcer",
    "mass",
    "lymph",
    "bleeding"
)
CUR_PATH = os.getcwd()
CFG_PATH = "/content/drive/MyDrive/Colab Notebooks/lesion_obd/Swin-Transformer-Object-Detection/configs/cascade_rcnn/cascade_rcnn_swin_tiny_fpn_1x_coco.py"
PREFIX = "/content/data/"
WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/lesion_obd/work_dir/version01"
# CHK_PATH = "/content/drive/MyDrive/Colab Notebooks/lesion_obd/cascade_mask_rcnn_swin_small_patch4_window7.pth"
# config file 들고오기
cfg = Config.fromfile(CFG_PATH)

In [ ]:
convert_to_coco_train(os.path.join(DATASET_PATH, 'train', 'train_label'),
        classes, coco_dict
)
convert_to_coco_valid(os.path.join(DATASET_PATH, 'train', 'train_label'),
        classes, coco_dict
)

CUR_PATH = os.getcwd()
CFG_PATH = os.path.join("/app/configs/cascade_rcnn/cascade_rcnn_swin_tiny_fpn_1x_coco.py")
PREFIX = os.path.join(DATASET_PATH, 'train', 'train_data')
WORK_DIR = os.path.join('/app/work_dir')

# config file 들고오기
cfg = Config.fromfile(CFG_PATH)

cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = CUR_PATH + "/all_train.json"

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = CUR_PATH + "/valid.json"

cfg.data.samples_per_gpu = opt.batch_size
cfg.data.workers_per_gpu = 4

cfg.seed = 42
cfg.gpu_ids = [0]

distributed = False
if distributed == True:
    init_dist('pytorch', **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)

cfg.work_dir = WORK_DIR
cfg.runner.max_epochs = 20
cfg.rtotal_epochs = 20
cfg.optimizer = dict(type='Adam', lr=opt.lr, weight_decay=0.0001)

cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=500, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

cfg.log_config.interval = 600
cfg.checkpoint_config.interval = 1
cfg.log_config = {'hooks': [{'type': 'TextLoggerHook'}], 'interval': 600}
cfg.model.pretrained = None

model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

In [ ]:
train_detector(model, datasets[0], cfg, distributed=distributed, validate=True)